# core

> All core functions and classes in ecfintools

In [ ]:
#| default_exp core

In [ ]:
#| hide
from eccore.ipython import nb_setup, pandas_nrows_ncols
from nbdev.showdoc import show_doc

In [ ]:
#| hide
nb_setup()

Set autoreload mode


In [ ]:
#| export
import re
from datetime import datetime
from pathlib import Path
from typing import Optional

import pandas as pd

# 

## Legacy function from former package `finutilities`

Set of functions that are transferred from my former package `finutilities`

In [ ]:
#| export
def safe_date(
    dt:str|datetime # date time to validate or convert
    ) -> datetime:  # datetime type object
    """Return a datetime object from string or datetime

    Accepted formats:
    
    - 'YYYY-MM-DD', 'YYYY/MM/DD', 'YYYY.MM.DD'
    - 'HH:MM:SS' can be added to the above formats
    
    """
    if isinstance(dt, datetime): return dt

    elif isinstance(dt, str):
        pattern = re.compile(r'\d{4}(?P<sep>[-\/\.])\d{2}(?P=sep)\d{2}(?P<t>\s?\d{2}:\d{2}:\d{2})?')
        match = pattern.search(dt)
        if match:
            sep = match.group('sep')
            t = match.group('t')
        else:
            msg = f"dt must be in format 'YYYY<sep>MM<sep>DD hh:mm:ss' where <sep> is one of '-/.'"
            raise AttributeError(msg)

        str_format = f"%Y{sep}%m{sep}%d{' %H:%M:%S' if t else ''}"
        return datetime.strptime(dt, str_format)

This function converts date-time strings into a `datetime` object, as long as the strings are in one of the following formats:

- `2024-12-15 21:02:17`
- `2024/12/15 21:02:17`
- `2024.12.15 21:02:17`
- `2024-12-15`
- `2024/12/15`
- `2024.12.15`

The function also accepts a datetime as argument `dt` and can be used to ensure a data argument used in a function is a `datetime` object.

In [ ]:
assert isinstance(safe_date('2024-12-15 21:02:17'), datetime)
assert isinstance(safe_date('2024/12/15 21:02:17'), datetime)
assert isinstance(safe_date('2024.12.15 21:02:17'), datetime)
assert isinstance(safe_date('2024-12-15'), datetime)
assert isinstance(safe_date('2024/12/15'), datetime)
assert isinstance(safe_date('2024.12.15'), datetime)
assert isinstance(safe_date(datetime.now()), datetime)

In [ ]:
#| export
def time_slice_df(
    df:pd.DataFrame,                   # `df` that will be sliced, must have a `DateTimeIndex`
    start:str|datetime|None = None,    # start date for slicing
    end:str|datetime|None = None       # end date for slicing
    ) -> pd.DataFrame:                 # sliced `df`
    """Slices DataFrame's rows from start to end, or the closest datetime available in the DataTimeIndex."""

    if not isinstance(df.index, pd.DatetimeIndex):
        raise AttributeError("df.index must be a DatetimeIndex")

    earliest, latest = df.index[0], df.index[-1]
    if start is None: start = earliest
    if end is None:  end = latest
    start, end = safe_date(start), safe_date(end)
    if start > end:
        msg = f"start ({start:%Y-%m-%d}) not before end ({end:%Y-%m-%d})"
        raise AttributeError(msg)

    sampled = df.loc[max(start, earliest):min(end, latest), :]
    return sampled

Even if we pass `start` and `end` dates that are not in the datafrane `DateTimeIndex`, the function still will slice using the closest datetime(s) in the index.

Let's take an test dataframe for illustration.

In [ ]:
#| export 
def load_test_df() -> pd.DataFrame:
    p2csv = Path('data-dev/ohlcv-data.csv')
    assert p2csv.is_file()
    df = pd.read_csv(p2csv, header=None, parse_dates=[[0,1]], index_col=0)
    df.columns = 'Open High Low Close Volume'.split()
    df.index.name = 'dt'
    return df

In [ ]:
df = load_test_df()
df.iloc[:15, :]

,Open,High,Low,Close,Volume
dt,,,,,
2018-10-22,2759.02,2779.27,2747.27,2754.48,26562
2018-10-23,2753.11,2755.36,2690.69,2743.45,38777
2018-10-24,2744.83,2748.58,2651.23,2672.80,41777
2018-10-25,2670.80,2722.90,2657.93,2680.71,39034
2018-10-26,2675.59,2692.34,2627.59,2663.57,61436
2018-10-29,2667.70,2707.00,2603.33,2639.17,44960
2018-10-30,2639.55,2689.50,2633.05,2688.50,52786
2018-10-31,2688.88,2736.76,2681.25,2704.75,32374
2018-11-01,2707.13,2741.58,2706.88,2731.90,29565


In this example, the DataFrame misses a few days, e.g.:
- 2018-10-27 and 2018-10-28
- 2018-11-03 and 2018-11-04

If we pick one of these dates for `start` and `end` respectively, we still get a sampled dataframe with the closed existing dates:

In [ ]:
time_slice_df(df, '2018-10-27', '2018-11-04')

,Open,High,Low,Close,Volume
dt,,,,,
2018-10-29,2667.70,2707.00,2603.33,2639.17,44960
2018-10-30,2639.55,2689.50,2633.05,2688.50,52786
2018-10-31,2688.88,2736.76,2681.25,2704.75,32374
2018-11-01,2707.13,2741.58,2706.88,2731.90,29565
2018-11-02,2725.28,2766.28,2699.96,2723.76,41892


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()